In [ ]:
import sys
import time
import numpy as np
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader

from datasets.dataset import SMPLRandomDataset
from models.SkinningNet import JointNet
from utils.LossFunction import JointLoss
from utils.OneRingIdx import getSkeletonOneRingIdx

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:%s"%device)

In [ ]:
batchSize = 16
trainDataset = SMPLRandomDataset(complexity = "skinning", 
                                 gender = "mixed", 
                                 dataSize = 5000, 
                                 vertexOrderRandom = True, 
                                 noise = 1, 
                                 rotate = False, 
                                 scale = True,
                                 translate = True,
                                 centre = False)
trainIter = DataLoader(trainDataset, num_workers=0, batch_size=batchSize, shuffle=True, drop_last=True)
allTrainNUM = len(trainDataset)

In [ ]:
net = JointNet()
net = net.to(device)

In [ ]:
lr = 0.00005
epochs = 100
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, epochs, eta_min=0.1*lr)
loss = JointLoss()

In [ ]:
beginTime = time.time()
net = net.train()

skeletonOneRingIdx = getSkeletonOneRingIdx("human")
skeletonOneRingIdx = torch.tensor(skeletonOneRingIdx).unsqueeze(0).repeat(batchSize, 1, 1).to(device).long()
for epoch in range(epochs):
    epochBegin = time.time()
    trainLoss = 0.0
    completedNum = 0
    batchNum = 0
    trainAcc = 0.0
    for V, facesOneRingIdx, rigW, joints in trainIter:
        V = V.to(device).float()
        facesOneRingIdx = facesOneRingIdx.to(device).long()
        rigW = rigW.to(device).float()
        joints = joints.to(device).float()
        opt.zero_grad()
        
        preJ = net(V, rigW, facesOneRingIdx, skeletonOneRingIdx)
        l = loss(preJ, joints)
        l.backward()
        opt.step()
        
        batchLoss = float(l)
        trainLoss += batchLoss
        batchNum += 1
        completedNum += batchSize
        compltePer = completedNum/allTrainNUM
        leftTime = ((time.time() - epochBegin)/compltePer - (time.time() - epochBegin))/60
        sys.stdout.write('\r Trainning %i / %i, joint loss: %f, percentage: %0.2f%%, remain %0.2f minuetes'%(completedNum, allTrainNUM, batchLoss, compltePer * 100, leftTime))
        sys.stdout.flush()

    save_path = './results/JointNet/'
    if epoch%5 == 0:
        torch.save(net.state_dict(), save_path + "%d.pkl"%epoch)
    torch.save(net.state_dict(), save_path + "latest.pkl")
    print("\n")
    print("Epoch%d,  joint loss: %f, time cost: %0.2f minuetes"%(epoch+1, trainLoss/batchNum, (time.time() - beginTime) / 60 ))
    print("----------------------------------")
    scheduler.step()